# O2O優惠卷使用預測

## 目標：
- 本賽題目標是預測投放的優惠券是否在規定時間內核銷
- 預測用戶領券後的使用情況，即用或者不用，轉化為二分類問題
- 針對此任務及一些相關背景知識，以該用戶使用於某日取得之優惠券核銷預測 AUC（ROC 曲線下面積）作為評價標準
- 即對將 User_id - Date_received - Coupon_id 為一組計算核銷預測的AUC值
- 若某使用者於同一日取得多張相同優惠券，則任一張核銷皆為有效核銷

## 時間區間：
- 訓練集是2016年1月1日到4月30日
- 測試集是2016年5月1日到6月15日
- Kaggle比賽是預測用戶在6月領取優惠卷後，15天內會使用的機率

# 載入資料

## 載入相關庫

In [1]:
#-*- coding=utf-8 -*-
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, log_loss, roc_auc_score, auc, roc_curve
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from IPython.display import Image, display
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline

from mlxtend.classifier import StackingClassifier

import pydotplus 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from datetime import date
import os, copy, time, re, pickle

import warnings
warnings.filterwarnings('ignore')

# display for this notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 讀csv數據

In [2]:
folder = './data/'
df_col_descript = pd.read_csv(folder + 'column_description_utf8.csv', encoding = 'utf8')
df_sample_submit = pd.read_csv(folder + 'sample_submission.csv')
df_off = pd.read_csv(folder + 'train_offline.csv')
df_test = pd.read_csv(folder + 'test_offline.csv')

## 概況

In [3]:
df_col_descript

,Column,Description
0,User_id,用戶 ID
1,Merchant_id,商家 ID
2,Coupon_id,優惠券 ID (null 代表無優惠券消費)
3,Discount_rate,"優惠券折價：[0,1] 代表折扣率；x:y 代表滿 x 減 y 元"
4,Distance,"用戶經常活動地點離商家最近距離 (x * 500 公尺), 0 表示低於 500 公尺, 1..."
5,Date_received,優惠券取得時間
6,Date,購買商品時間 (如果 Date is null & Coupon_id is not nul...


- User_id：用戶 ID 
- Merchant_id：商家 ID 
- Coupon_id：優惠券 ID (null 代表無優惠券消費) 
- Discount_rate：優惠券折價：[0,1] 代表折扣率；x:y 代表滿 x 減 y 元 
- Distance：用戶經常活動地點離商家最近距離 (x * 500 公尺), 0 表示低於 500 公尺, 10 表示大於 5 公里。 
- Date_received：優惠券取得時間。 
- Date：購買商品時間 (如果 Date is null & Coupon_id is not null, 則該紀錄為有優惠券但未使用; 若為 Date is not null & Coupon_id is null, 則為普通消費日期; 若 Date is not null & Coupon_id is not null, 則表示優惠券消費日期)

In [4]:
print(df_sample_submit.shape)
print(df_sample_submit['label'][df_sample_submit['label']>0.5].count())
df_sample_submit.head()

(304096, 2)
152068


,uid,label
0,1000020_2705_20160519,0.397000
1,1000020_8192_20160513,0.059435
2,1000065_1455_20160527,0.087798
3,1000085_8067_20160513,0.806111
4,1000086_2418_20160613,0.825684


In [5]:
print(df_off.shape)
df_off.head()

(1160742, 7)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN


In [6]:
print(df_test.shape)
df_test.head()

(594142, 6)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,1439408,4663,11002.0,150:20,1.0,20160528.0
1,1439408,2632,8591.0,20:1,0.0,20160613.0
2,1439408,2632,NaN,NaN,0.0,NaN
3,1439408,2632,8591.0,20:1,0.0,20160516.0
4,2029232,450,1532.0,30:5,0.0,20160530.0


## 資料型態

In [7]:
def info_type(df):
    for col in df.columns:
        print(col, df[col].dtype)

In [8]:
info_type(df_off)
print('\n--\n')
info_type(df_test)

User_id int64
Merchant_id int64
Coupon_id float64
Discount_rate object
Distance float64
Date_received float64
Date float64

--

User_id int64
Merchant_id int64
Coupon_id float64
Discount_rate object
Distance float64
Date_received float64


In [9]:
# 把原始數據型態轉成 str

def convert_all_type_to_str(df):
    for col in df.columns:
        df[col] = df[col].apply(str)
    return df

def remove_dot_zero(row):
    if '.' in row:
        strs = row.split('.')
        return str(strs[0])
    else:
        return row

def remove_cols_dot_zero(df, cols):
    for col in cols:
        df[col] = df[col].apply(remove_dot_zero)
    return df

In [10]:
df_off = convert_all_type_to_str(df_off)
df_test = convert_all_type_to_str(df_test)

cols_dot_zero = ['Coupon_id', 'Date_received']
df_off = remove_cols_dot_zero(df_off, cols_dot_zero + ['Date'])
df_test = remove_cols_dot_zero(df_test, cols_dot_zero)

In [11]:
df_off.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,nan,nan,0.0,nan,20160217
1,1439408,2632,8591,20:1,0.0,20160217,nan
2,1439408,2632,1078,20:1,0.0,20160319,nan
3,1832624,3381,7610,200:20,0.0,20160429,nan
4,2029232,3381,11951,200:20,1.0,20160129,nan


## 優惠卷使用狀況

In [12]:
def info_discount_use(df):
    print('有優惠卷，購買商品：%d' % df[(df['Date_received'] != 'nan') & (df['Date'] != 'nan')].shape[0])
    print('有優惠卷，未購商品：%d' % df[(df['Date_received'] != 'nan') & (df['Date'] == 'nan')].shape[0])
    print('無優惠卷，購買商品：%d' % df[(df['Date_received'] == 'nan') & (df['Date'] != 'nan')].shape[0])
    print('無優惠卷，未購商品：%d' % df[(df['Date_received'] == 'nan') & (df['Date'] == 'nan')].shape[0])

In [13]:
info_discount_use(df_off)

有優惠卷，購買商品：42936
有優惠卷，未購商品：704033
無優惠卷，購買商品：413773
無優惠卷，未購商品：0


- 很多人（413773）購買商品卻沒有使用優惠券
- 也有很多人（704033）有優惠券但卻沒有使用
- 真正使用優惠券購買商品的人（42936）很少！
- 所以，這個比賽表面意義是推測投放的優惠卷是否會被使用，真正的意義是把優惠券送給真正可能會購買商品的人

# 特徵工程

特徵工程比選擇哪種算法更加重要<br/>

## 優惠卷折率 Discount_rate
一般情況下優惠得越多，用戶就越有可能使用優惠券

In [14]:
def info_discount_type(df):
    print('Discount_rate 類型：\n',df['Discount_rate'].unique())

In [15]:
info_discount_type(df_off)

Discount_rate 類型：
 ['nan' '20:1' '200:20' '10:5' '100:10' '200:30' '20:5' '30:5' '50:10'
 '150:10' '100:30' '200:50' '100:50' '300:30' '50:20' '0.9' '50:5'
 '150:20' '10:1' '30:10' '30:1' '0.95' '100:5' '50:1' '100:20' '0.8'
 '300:20' '100:1' '20:10' '0.85' '0.6' '5:1' '150:30' '200:10' '300:50'
 '0.5' '200:5' '0.7' '0.75' '30:20' '0.2' '150:50' '300:10' '50:30'
 '150:5' '200:100']


打折率分為 3 種情況：
- null 表示沒有打折
- [0,1] 表示折扣率
- x:y 表示滿 x 減 y

構建4種特徵：
- 打折類型
- 折扣率
- 滿多少
- 減多少

In [16]:
def get_discount_type(row):
    if row == 'nan':
        return -1
    elif ':' in row:
        return 1
    else:
        return 0

def convert_discount_to_rate(row):
    if row == 'nan':
        return 1.0
    elif ':' in row:
        strs = row.split(':')
        return 1.0 - float(strs[1])/float(strs[0])
    else:
        return float(row)
    
def get_discount_top_up(row):
    if ':' in row:
        strs = row.split(':')
        return float(strs[0])
    else:
        return 0
    
def get_discount_off(row):
    if ':' in row:
        strs = row.split(':')
        return float(strs[1])
    else:
        return 0

def process_Discount_rate(df):
    df['discount_type'] = df['Discount_rate'].apply(get_discount_type)
    df['discount_rate'] = df['Discount_rate'].apply(convert_discount_to_rate)
    df['discount_top_up'] = df['Discount_rate'].apply(get_discount_top_up)
    df['discount_off'] = df['Discount_rate'].apply(get_discount_off)
    
    print(df['discount_rate'].unique())
    return df

In [17]:
df_off = process_Discount_rate(df_off)
df_test = process_Discount_rate(df_test)

[1.         0.95       0.9        0.5        0.85       0.75
 0.83333333 0.8        0.93333333 0.7        0.6        0.86666667
 0.66666667 0.96666667 0.98       0.99       0.975      0.33333333
 0.2        0.4       ]
[0.86666667 0.95       1.         0.83333333 0.8        0.75
 0.66666667 0.9        0.96666667 0.6        0.7        0.5
 0.98       0.85       0.93333333 0.99       0.975      0.33333333]


In [18]:
df_off.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_top_up,discount_off
0,1439408,2632,nan,nan,0.0,nan,20160217,-1,1.00,0.0,0.0
1,1439408,2632,8591,20:1,0.0,20160217,nan,1,0.95,20.0,1.0
2,1439408,2632,1078,20:1,0.0,20160319,nan,1,0.95,20.0,1.0
3,1832624,3381,7610,200:20,0.0,20160429,nan,1,0.90,200.0,20.0
4,2029232,3381,11951,200:20,1.0,20160129,nan,1,0.90,200.0,20.0


## 距離 Distance
距離的遠近也會影響到優惠券的使用與否

In [19]:
def info_distance(df):
    print(df['Distance'].unique())

In [20]:
info_distance(df_off)

['0.0' '1.0' '2.0' 'nan' '10.0' '4.0' '7.0' '9.0' '3.0' '5.0' '6.0' '8.0']


In [21]:
# 提取特徵distance，並將距離 str 轉為 int

def process_Distance(df):
    df['distance'] = df['Distance'].replace('nan', -1).astype(float)
    print(df['distance'].unique())
    return df

In [22]:
df_off = process_Distance(df_off)
df_test = process_Distance(df_test)

[ 0.  1.  2. -1. 10.  4.  7.  9.  3.  5.  6.  8.]
[ 1.  0. -1. 10.  7.  2.  9.  4.  8.  3.  6.  5.]


In [23]:
df_off.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_top_up,discount_off,distance
0,1439408,2632,nan,nan,0.0,nan,20160217,-1,1.00,0.0,0.0,0.0
1,1439408,2632,8591,20:1,0.0,20160217,nan,1,0.95,20.0,1.0,0.0
2,1439408,2632,1078,20:1,0.0,20160319,nan,1,0.95,20.0,1.0,0.0
3,1832624,3381,7610,200:20,0.0,20160429,nan,1,0.90,200.0,20.0,0.0
4,2029232,3381,11951,200:20,1.0,20160129,nan,1,0.90,200.0,20.0,1.0


## 領劵日期 Date_received

一般而言，週末領取優惠券去消費的可能性更大一些

In [24]:
def info_received_date_and_buy_date(df):
    date_received = df['Date_received'].unique()
    date_received = sorted(date_received[date_received != 'nan'])

    date_buy = df['Date'].unique()
    date_buy = sorted(date_buy[date_buy != 'nan'])

    print('優惠卷收到日期從',date_received[0],'到',date_received[-1])
    print('消費日期從',date_buy[0],'到',date_buy[-1])

In [25]:
info_received_date_and_buy_date(df_off)

優惠卷收到日期從 20160101 到 20160430
消費日期從 20160101 到 20160628


領劵日期的特徵：
- weekday : {null, 1, 2, 3, 4, 5, 6, 7}
- weekday_type : {1, 0}（週六和周日為1，其他為0）
- Weekda_1 : {1, 0, 0, 0, 0, 0, 0}
- Weekday_2 : {0, 1, 0, 0, 0, 0, 0}
- Weekday_3 : {0, 0, 1, 0, 0, 0, 0}
- Weekday_4 : {0, 0, 0, 1, 0, 0, 0}
- Weekday_5 : {0, 0, 0, 0, 1, 0, 0}
- Weekday_6 : {0, 0, 0, 0, 0, 1, 0}
- Weekday_7 : {0, 0, 0, 0, 0, 0, 1}

In [26]:
def get_weekday(row):
   if row == 'nan':
       return np.nan
   else:
       # add one to make it from 0~6 -> 1~7
       return int(date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday() + 1)

weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
#print(weekdaycols)

def get_weekday_type(row):
    if row == 'nan':
        return -1
    elif row in [6,7]:
        return 1
    else:
        return 0

def process_Date_received(df):
    df['weekday'] = df['Date_received'].apply(get_weekday)

    # weekday_type :  週六和周日為1，其他為0
    df['weekday_type'] = df['weekday'].apply(get_weekday_type).astype(int)
    
    # change weekday to one-hot encoding 
    df_ = pd.get_dummies(df['weekday'].replace('nan', np.nan))
    df_.columns = weekdaycols
    df[weekdaycols] = df_
    
    df['weekday'] = df['weekday'].replace(np.nan, -1)
    
    return df

In [27]:
df_off = process_Date_received(df_off)
df_test = process_Date_received(df_test)

In [28]:
df_off.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_top_up,...,distance,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,1439408,2632,nan,nan,0.0,nan,20160217,-1,1.00,0.0,...,0.0,-1.0,0,0,0,0,0,0,0,0
1,1439408,2632,8591,20:1,0.0,20160217,nan,1,0.95,20.0,...,0.0,3.0,0,0,0,1,0,0,0,0
2,1439408,2632,1078,20:1,0.0,20160319,nan,1,0.95,20.0,...,0.0,6.0,1,0,0,0,0,0,1,0
3,1832624,3381,7610,200:20,0.0,20160429,nan,1,0.90,200.0,...,0.0,5.0,0,0,0,0,0,1,0,0
4,2029232,3381,11951,200:20,1.0,20160129,nan,1,0.90,200.0,...,1.0,5.0,0,0,0,0,0,1,0,0


## 標注標籤 Label

三種情況：
- 沒有領到優惠券，無需考慮，y = -1
- 領取優惠券且在15天內使用，即正樣本，y = 1
- 領取優惠券未在在15天內使用，即負樣本，y = 0

In [29]:
def label(row):
   if row['Date_received'] == 'nan':
       return -1
   if row['Date'] != 'nan':
       td = pd.to_datetime(row['Date'], format='%Y%m%d') - pd.to_datetime(row['Date_received'], format='%Y%m%d')
       if td <= pd.Timedelta(15, 'D'):
           return 1
   return 0

def process_label(df):
    df['label'] = df.apply(label, axis=1)
    return df

In [30]:
df_off = process_label(df_off)

In [31]:
df_off['label'].value_counts()

 0    710665
-1    413773
 1     36304
Name: label, dtype: int64

In [32]:
df_off.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_top_up,...,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,label
0,1439408,2632,nan,nan,0.0,nan,20160217,-1,1.00,0.0,...,-1.0,0,0,0,0,0,0,0,0,-1
1,1439408,2632,8591,20:1,0.0,20160217,nan,1,0.95,20.0,...,3.0,0,0,0,1,0,0,0,0,0
2,1439408,2632,1078,20:1,0.0,20160319,nan,1,0.95,20.0,...,6.0,1,0,0,0,0,0,1,0,0
3,1832624,3381,7610,200:20,0.0,20160429,nan,1,0.90,200.0,...,5.0,0,0,0,0,0,1,0,0,0
4,2029232,3381,11951,200:20,1.0,20160129,nan,1,0.90,200.0,...,5.0,0,0,0,0,0,1,0,0,0


- 正樣本共有 36304 例
- 負樣本共有 710665 例
- 正負樣本數量差別很大，這也是為什麼會使用 AUC 作為模型性能評估標準的原因

# 模型

In [33]:
def info_date_received(df):
    date_vc = df['Date_received'].value_counts().sort_index(ascending=True)
    print(date_vc.head(2))
    print(date_vc.tail(2))

In [34]:
info_date_received(df_off)
print('\n--\n')
info_date_received(df_test)

20160101    554
20160102    542
Name: Date_received, dtype: int64
20160430      5008
nan         413773
Name: Date_received, dtype: int64

--

20160501    3121
20160502    3276
Name: Date_received, dtype: int64
20160615      3475
nan         287829
Name: Date_received, dtype: int64


## 劃分訓練集/驗證集

劃分訓練集/驗證集，劃分方式是按照領券日期：
- 訓練集：20160101-20160331
- 驗證集：20160401-20160430

其他資訊：
- 收卷日期從 20160101-20160430
- 消費日期從 20160101-20160628
- df_train是2016年1月1日到4月30日
- df_test是2016年5月1日到6月15日
- Kaggle預測用戶在6月領取優惠卷後，15天內會使用的機率

In [35]:
def process_split_train_valid_set(df):
    df_ = df[df['label'] != -1].copy()
    train = df_[(df_['Date_received'] < '20160416')].copy()
    valid = df_[(df_['Date_received'] >= '20160416') & (df_['Date_received'] <= '20160430')].copy()
    print('Train Set: \n', train['label'].value_counts())
    print('Valid Set: \n', valid['label'].value_counts())
    return train, valid

In [36]:
df_train, df_valid = process_split_train_valid_set(df_off)

Train Set: 
 0    635281
1     32472
Name: label, dtype: int64
Valid Set: 
 0    75384
1     3832
Name: label, dtype: int64


## 特徵數量

In [37]:
# feature
original_feature = ['discount_rate','discount_type','discount_top_up', 'discount_off','distance', 'weekday', 'weekday_type'] + weekdaycols
print('共有特徵：',len(original_feature),'個')
print(original_feature)

共有特徵： 14 個
['discount_rate', 'discount_type', 'discount_top_up', 'discount_off', 'distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


## 建立模型

In [38]:
def check_model_SGDClassifier(X_train, Y_train):
# 模型採用的是 SGDClassifier：
# 使用了 Python 中的 Pipeline 管道機制
# 可以使參數集在新數據集（比如測試集）上的重復使用
# 管道機制實現了對全部步驟的流式化封裝和管理。

# 交叉驗證採用 StratifiedKFold：
# 其用法類似 Kfold，但是 StratifiedKFold 是分層採樣
# 確保訓練集、測試集中各類別樣本的比例與原始數據集中相同
   classifier = lambda: SGDClassifier(
       loss='log',  # loss function: logistic regression
       penalty='elasticnet', # L1 & L2
       fit_intercept=True,  # 是否存在截距，默認存在
       max_iter=100, 
       shuffle=True,  # Whether or not the training data should be shuffled after each epoch
       n_jobs=1, # The number of processors to use
       class_weight=None) # Weights associated with classes. If not given, all classes are supposed to have weight one.

   # 管道機制使得參數集在新數據集（比如測試集）上的重復使用，管道機制實現了對全部步驟的流式化封裝和管理。
   model = Pipeline(steps=[
       ('ss', StandardScaler()), # transformer
       ('en', classifier())  # estimator
   ])

   parameters = {
       'en__alpha': [ 0.001, 0.01, 0.1],
       'en__l1_ratio': [ 0.001, 0.01, 0.1]
   }

   # StratifiedKFold用法類似Kfold，但是他是分層採樣，確保訓練集，測試集中各類別樣本的比例與原始數據集中相同。
   folder = StratifiedKFold(n_splits=3, shuffle=True)
   
   # Exhaustive search over specified parameter values for an estimator.
   grid_search = GridSearchCV(
       model, 
       parameters, 
       cv=folder, 
       n_jobs=-1,  # -1 means using all processors
       verbose=1)
   grid_search = grid_search.fit(X_train, Y_train)
   
   return grid_search

def check_model_LogisticRegression(X_train, Y_train):
    lr = LogisticRegression(tol=0.001, penalty='l2', fit_intercept=True, C=1.0)
    lr.fit(X_train, Y_train)
    return lr
    
def check_model_GradientBoostingClassifier(X_train, Y_train):
    gdbt = GradientBoostingClassifier(tol=100, subsample=0.75, n_estimators=250, max_features=20,
                                  max_depth=6, learning_rate=0.03)
    gdbt.fit(X_train, Y_train)
    return gdbt

def check_model_RandomForestClassifier(X_train, Y_train):
    rf = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=1, 
                            max_features='sqrt', max_depth=6, bootstrap=True)
    rf.fit(X_train, Y_train)
    return rf

def check_model_StackingClassifier(X_train, Y_train):
    lr = check_model_LogisticRegression(X_train, Y_train)
    gdbt = check_model_GradientBoostingClassifier(X_train, Y_train)
    rf = check_model_RandomForestClassifier(X_train, Y_train)
    
    meta_estimator = GradientBoostingClassifier(tol=100, subsample=0.70, n_estimators=50, 
                                               max_features='sqrt', max_depth=4, learning_rate=0.3)
    model = StackingClassifier(classifiers=[lr, gdbt, rf], meta_classifier=meta_estimator)

    model.fit(X_train, Y_train)
    return model

## 檢查nan, infinite錯誤

In [39]:
def check_is_any_nan_and_infinite(df):
    for col in df.columns:
        any_nan = np.any(np.isnan(df[col])) # should gets False
        all_finite = np.all(np.isfinite(df[col])) # should gets True
        if any_nan == True or all_finite == False:
            print(col, df[col].dtype, any_nan, all_finite)

## 計算AUC

In [40]:
def cal_auc_and_acc(df_valid, y_valid_pred):
    auc_score = roc_auc_score(y_true=df_valid['label'], y_score=df_valid['pred_prob'])
    acc = accuracy_score(y_true=df_valid['label'], y_pred=y_valid_pred.argmax(axis=1))
    print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))
    
def cal_avg_auc(df):
    gb = df.groupby(['Coupon_id'])
    aucs = []
    for i in gb:
        df_ = i[1] 
        if len(df_['label'].unique()) != 2:
            continue
        fpr, tpr, thresholds = roc_curve(df_['label'], df_['pred_prob'], pos_label=1)
        aucs.append(auc(fpr, tpr))
    print('AvgAUC:', np.average(aucs))

## 驗證
- 對驗證集中每個優惠券預測的結果計算 AUC，再對所有優惠券的 AUC 求平均
- 計算 AUC 的時候，如果 label 只有一類，就直接跳過，因為 AUC 無法計算

In [41]:
def check_valid(model, df, predics):
    y_valid_pred = model.predict_proba(df[predics])
    df_valid = df.copy()
    df_valid['pred_prob'] = y_valid_pred[:, 1]
    cal_auc_and_acc(df_valid, y_valid_pred)
    cal_avg_auc(df_valid)
    df_valid.head()

## 預測測試集

In [42]:
def pred_test(model, df, predicts):
    y_pred = model.predict_proba(df[predicts])
    df_ = df[['User_id','Date_received', 'Coupon_id']].copy()
    df_['label'] = y_pred[:,1]
    return df_

def edit_proper_column(df):
    df['uid'] = df['User_id']+'_'+df['Coupon_id']+'_'+df['Date_received']
    df = df[(df['Date_received'] != 'nan') | (df['Coupon_id'] != 'nan')]
    df = df.drop(['User_id', 'Date_received', 'Coupon_id'], axis=1)
    return df

def keep_higher_probability_if_same_uid(df):
    df = df.sort_values('label', ascending=False)
    df = df.drop_duplicates(subset=['uid'], keep='first')
    return df

def process_pred(model, df, predicts):
    df_ = pred_test(model, df, predicts)
    df_ = edit_proper_column(df_)
    df_ = keep_higher_probability_if_same_uid(df_)
    return df_

## 輸出 submition csv

In [43]:
def to_csv(df, file_name):
    df.to_csv(file_name, index=False, header=True)

## 保存/讀取 Model

In [44]:
def save_model(model, model_name):
    with open(model_name, 'wb') as f:
        pickle.dump(model, f)

def load_model(model_name):
    with open(model_name, 'rb') as f:
        model = pickle.load(f)

In [45]:
predictors = original_feature
check_is_any_nan_and_infinite(df_test[predictors])
X_train = df_train[predictors]
Y_train = df_train['label']

## 訓練到輸出

In [46]:
model = check_model_SGDClassifier(X_train, Y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    9.8s finished


In [47]:
check_valid(model, df_valid, predictors)

df_submit = process_pred(model, df_test, predictors)

to_csv(df_submit, 'submission_SGDClassifier.csv')
save_model(model, 'SGDClassifier_model.pkl')

Validation AUC: 0.756, Accuracy: 0.952
AvgAUC: 0.53353689353856


# 優化模型

## 特征提取

In [48]:
fdf = df_train.copy()

### 用戶 User 特徵

In [49]:
# u：用戶統計
u = fdf[['User_id']].copy().drop_duplicates()

In [50]:
# u1:用戶接收到優惠券的數量
u1 = fdf[fdf['Date_received'] != 'null'][['User_id']].copy()
u1['u_coupon_count'] = 1
u1 = u1.groupby(['User_id'], as_index = False).count()
u1.head()

,User_id,u_coupon_count
0,1000026,5
1,100004,1
2,1000076,2
3,1000086,2
4,1000093,1


In [51]:
# u2：用戶購買的次數
# u_buy_count : times of user buy offline (with or without coupon)
u2 = fdf[fdf['Date'] != 'null'][['User_id']].copy()
u2['u_buy_count'] = 1
u2 = u2.groupby(['User_id'], as_index = False).count()
u2.head()

,User_id,u_buy_count
0,1000026,5
1,100004,1
2,1000076,2
3,1000086,2
4,1000093,1


In [52]:
# u3：用戶使用優惠券購買的次數
# u_buy_with_coupon : times of user buy offline (with coupon)
u3 = fdf[((fdf['Date'] != 'null') & (fdf['Date_received'] != 'null'))][['User_id']].copy()
u3['u_buy_with_coupon'] = 1
u3 = u3.groupby(['User_id'], as_index = False).count()
u3.head()

,User_id,u_buy_with_coupon
0,1000026,5
1,100004,1
2,1000076,2
3,1000086,2
4,1000093,1


In [53]:
# u4：用戶購買的商家個數
# u_merchant_count : num of merchant user bought from
u4 = fdf[fdf['Date'] != 'null'][['User_id', 'Merchant_id']].copy()
u4.drop_duplicates(inplace = True)
u4 = u4.groupby(['User_id'], as_index = False).count()
u4.rename(columns = {'Merchant_id':'u_merchant_count'}, inplace = True)
u4.head()

,User_id,u_merchant_count
0,1000026,4
1,100004,1
2,1000076,2
3,1000086,1
4,1000093,1


In [54]:
# u5：用戶使用優惠券購買商品距離商店的最小距離
# u6：用戶使用優惠券購買商品距離商店的最大距離
# u7：用戶使用優惠券購買商品距離商店的平均距離
# u8：用戶使用優惠券購買商品距離商店的中位數距離
# u_min_distance
utmp = fdf[(fdf['Date'] != 'null') & (fdf['Date_received'] != 'null')][['User_id', 'distance']].copy()
utmp.replace(-1, np.nan, inplace = True)
u5 = utmp.groupby(['User_id'], as_index = False).min()
u5.rename(columns = {'distance':'u_min_distance'}, inplace = True)
u6 = utmp.groupby(['User_id'], as_index = False).max()
u6.rename(columns = {'distance':'u_max_distance'}, inplace = True)
u7 = utmp.groupby(['User_id'], as_index = False).mean()
u7.rename(columns = {'distance':'u_mean_distance'}, inplace = True)
u8 = utmp.groupby(['User_id'], as_index = False).median()
u8.rename(columns = {'distance':'u_median_distance'}, inplace = True)
u8.head()

,User_id,u_median_distance
0,1000026,0.0
1,100004,8.0
2,1000076,10.0
3,1000086,0.0
4,1000093,10.0


In [55]:
# 根據 User_id，將 u1,u2,u3,u4,u5,u6,u7,u8 整合成 user_feature
# merge all the features on key User_id
user_feature = pd.merge(u, u1, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u2, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u3, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u4, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u5, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u6, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u7, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u8, on = 'User_id', how = 'left')

In [56]:
# u_use_coupon_rate：對於用戶來說，接收到的優惠券使用率
# u_buy_with_coupon_rate：用戶所有購買行為中使用優惠券佔的比例
# calculate rate
user_feature['u_use_coupon_rate'] = user_feature['u_buy_with_coupon'].astype('float')/user_feature['u_coupon_count'].astype('float')
user_feature['u_buy_with_coupon_rate'] = user_feature['u_buy_with_coupon'].astype('float')/user_feature['u_buy_count'].astype('float')
user_feature = user_feature.fillna(0)
user_feature.head()

,User_id,u_coupon_count,u_buy_count,u_buy_with_coupon,u_merchant_count,u_min_distance,u_max_distance,u_mean_distance,u_median_distance,u_use_coupon_rate,u_buy_with_coupon_rate
0,1439408,2,2,2,1,0.0,0.0,0.0,0.0,1.0,1.0
1,2029232,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0
2,2223968,1,1,1,1,2.0,2.0,2.0,2.0,1.0,1.0
3,73611,1,1,1,1,0.0,0.0,0.0,0.0,1.0,1.0
4,3273056,1,1,1,1,10.0,10.0,10.0,10.0,1.0,1.0


### 商家 Merchant 特徵

In [57]:
# m：商家統計
# key of merchant
m = fdf[['Merchant_id']].copy().drop_duplicates()

In [58]:
# m1：每個商家發放的優惠券數量
# m_coupon_count : num of coupon from merchant
m1 = fdf[fdf['Date_received'] != 'null'][['Merchant_id']].copy()
m1['m_coupon_count'] = 1
m1 = m1.groupby(['Merchant_id'], as_index = False).count()
m1.head()

,Merchant_id,m_coupon_count
0,1002,8
1,1007,3
2,1016,3
3,1028,6
4,1029,4


In [59]:
# m2：商家銷售的次數（不考慮是否使用優惠券）
# m_sale_count : num of sale from merchant (with or without coupon)
m2 = fdf[fdf['Date'] != 'null'][['Merchant_id']].copy()
m2['m_sale_count'] = 1
m2 = m2.groupby(['Merchant_id'], as_index = False).count()
m2.head()

,Merchant_id,m_sale_count
0,1002,8
1,1007,3
2,1016,3
3,1028,6
4,1029,4


In [60]:
# m3：商家使用優惠券銷售的次數
# m_sale_with_coupon : num of sale from merchant with coupon usage
m3 = fdf[(fdf['Date'] != 'null') & (fdf['Date_received'] != 'null')][['Merchant_id']].copy()
m3['m_sale_with_coupon'] = 1
m3 = m3.groupby(['Merchant_id'], as_index = False).count()
m3.head()

,Merchant_id,m_sale_with_coupon
0,1002,8
1,1007,3
2,1016,3
3,1028,6
4,1029,4


In [61]:
# m4：商家銷售的用戶個數
# u_merchant_count : num of merchant user bought from
m4 = fdf[fdf['Date'] != 'null'][['User_id', 'Merchant_id']].copy()
m4.drop_duplicates(inplace = True)
m4 = m4.groupby(['Merchant_id'], as_index = False).count()
m4.rename(columns = {'User_id':'m_user_count'}, inplace = True)
m4.head()

,Merchant_id,m_user_count
0,1002,4
1,1007,3
2,1016,3
3,1028,4
4,1029,4


In [62]:
# m5：商家使用優惠券銷售商品距離用戶的最小距離
# m6：商家使用優惠券銷售商品距離用戶的最大距離
# m7：商家使用優惠券銷售商品距離用戶的平均距離
# m8：商家使用優惠券銷售商品距離用戶的中位數距離
# m_min_distance
mtmp = fdf[(fdf['Date'] != 'null') & (fdf['Date_received'] != 'null')][['Merchant_id', 'distance']].copy()
mtmp.replace(-1, np.nan, inplace = True)
m5 = mtmp.groupby(['Merchant_id'], as_index = False).min()
m5.rename(columns = {'distance':'m_min_distance'}, inplace = True)
m6 = mtmp.groupby(['Merchant_id'], as_index = False).max()
m6.rename(columns = {'distance':'m_max_distance'}, inplace = True)
m7 = mtmp.groupby(['Merchant_id'], as_index = False).mean()
m7.rename(columns = {'distance':'m_mean_distance'}, inplace = True)
m8 = mtmp.groupby(['Merchant_id'], as_index = False).median()
m8.rename(columns = {'distance':'m_median_distance'}, inplace = True)
m8.head()

,Merchant_id,m_median_distance
0,1002,0.0
1,1007,4.0
2,1016,2.0
3,1028,5.0
4,1029,0.0


In [63]:
# 根據 Merchant_id，將 m1,m2,m3,m4,m5,m6,m7,m8 整合成 merchant_feature
merchant_feature = pd.merge(m, m1, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m2, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m3, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m4, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m5, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m6, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m7, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m8, on = 'Merchant_id', how = 'left')
merchant_feature = merchant_feature.fillna(0)
merchant_feature.head()

,Merchant_id,m_coupon_count,m_sale_count,m_sale_with_coupon,m_user_count,m_min_distance,m_max_distance,m_mean_distance,m_median_distance
0,2632,30,30,30,10,0.0,10.0,1.227273,1.0
1,3381,103211,103211,103211,88214,0.0,10.0,2.974950,1.0
2,2099,16824,16824,16824,12820,0.0,10.0,2.435773,1.0
3,4833,8321,8321,8321,7949,0.0,10.0,7.363286,10.0
4,8390,690,690,690,441,0.0,10.0,0.710652,0.0


In [64]:
# m_coupon_use_rate： 對於商家來說，發放的優惠卷使用率
# m_sale_with_coupon_rate：商家所有銷售行為中使用優惠卷佔的比例
merchant_feature['m_coupon_use_rate'] = merchant_feature['m_sale_with_coupon'].astype('float')/merchant_feature['m_coupon_count'].astype('float')
merchant_feature['m_sale_with_coupon_rate'] = merchant_feature['m_sale_with_coupon'].astype('float')/merchant_feature['m_sale_count'].astype('float')
merchant_feature = merchant_feature.fillna(0)
merchant_feature.head()

,Merchant_id,m_coupon_count,m_sale_count,m_sale_with_coupon,m_user_count,m_min_distance,m_max_distance,m_mean_distance,m_median_distance,m_coupon_use_rate,m_sale_with_coupon_rate
0,2632,30,30,30,10,0.0,10.0,1.227273,1.0,1.0,1.0
1,3381,103211,103211,103211,88214,0.0,10.0,2.974950,1.0,1.0,1.0
2,2099,16824,16824,16824,12820,0.0,10.0,2.435773,1.0,1.0,1.0
3,4833,8321,8321,8321,7949,0.0,10.0,7.363286,10.0,1.0,1.0
4,8390,690,690,690,441,0.0,10.0,0.710652,0.0,1.0,1.0


### 用戶和商家 User & Merchant 交叉特徵

In [65]:
# um1：用戶和商家對統計
# key of user and merchant
um = fdf[['User_id', 'Merchant_id']].copy().drop_duplicates()

In [66]:
um1 = fdf[['User_id', 'Merchant_id']].copy()
um1['um_count'] = 1
um1 = um1.groupby(['User_id', 'Merchant_id'], as_index = False).count()
um1.head()

,User_id,Merchant_id,um_count
0,1000026,1569,1
1,1000026,3381,1
2,1000026,450,1
3,1000026,7221,2
4,100004,6284,1


In [67]:
# um2：每個用戶商家對交易統計
um2 = fdf[fdf['Date'] != 'null'][['User_id', 'Merchant_id']].copy()
um2['um_buy_count'] = 1
um2 = um2.groupby(['User_id', 'Merchant_id'], as_index = False).count()
um2.head()

,User_id,Merchant_id,um_buy_count
0,1000026,1569,1
1,1000026,3381,1
2,1000026,450,1
3,1000026,7221,2
4,100004,6284,1


In [68]:
# um3：每個用戶商家對發放優惠券的統計
um3 = fdf[fdf['Date_received'] != 'null'][['User_id', 'Merchant_id']].copy()
um3['um_coupon_count'] = 1
um3 = um3.groupby(['User_id', 'Merchant_id'], as_index = False).count()
um3.head()

,User_id,Merchant_id,um_coupon_count
0,1000026,1569,1
1,1000026,3381,1
2,1000026,450,1
3,1000026,7221,2
4,100004,6284,1


In [69]:
# um4：每個用戶商家對使用優惠卷的交易行為的統計
um4 = fdf[(fdf['Date_received'] != 'null') & (fdf['Date'] != 'null')][['User_id', 'Merchant_id']].copy()
um4['um_buy_with_coupon'] = 1
um4 = um4.groupby(['User_id', 'Merchant_id'], as_index = False).count()
um4.head()

,User_id,Merchant_id,um_buy_with_coupon
0,1000026,1569,1
1,1000026,3381,1
2,1000026,450,1
3,1000026,7221,2
4,100004,6284,1


In [70]:
# 根據 User_id 和 Merchant_id，將 um1,um2,um3,um4 整合成 user_merchant_feature
# merge all user merchant 
user_merchant_feature = pd.merge(um, um1, on = ['User_id','Merchant_id'], how = 'left')
user_merchant_feature = pd.merge(user_merchant_feature, um2, on = ['User_id','Merchant_id'], how = 'left')
user_merchant_feature = pd.merge(user_merchant_feature, um3, on = ['User_id','Merchant_id'], how = 'left')
user_merchant_feature = pd.merge(user_merchant_feature, um4, on = ['User_id','Merchant_id'], how = 'left')
user_merchant_feature = user_merchant_feature.fillna(0)

In [71]:
# um_buy_rate：每個用戶商家對交易行為佔所有用戶商戶對的比例
# um_coupon_use_rate：使用優惠卷的交易行為佔所有用戶商家對發放優惠卷的比例
# um_buy_with_coupon_rate：使用優惠卷的交易行為佔所有用戶商家對交易行為的比例
user_merchant_feature['um_buy_rate'] = user_merchant_feature['um_buy_count'].astype('float')/user_merchant_feature['um_count'].astype('float')
user_merchant_feature['um_coupon_use_rate'] = user_merchant_feature['um_buy_with_coupon'].astype('float')/user_merchant_feature['um_coupon_count'].astype('float')
user_merchant_feature['um_buy_with_coupon_rate'] = user_merchant_feature['um_buy_with_coupon'].astype('float')/user_merchant_feature['um_buy_count'].astype('float')
user_merchant_feature = user_merchant_feature.fillna(0)
user_merchant_feature.head()

,User_id,Merchant_id,um_count,um_buy_count,um_coupon_count,um_buy_with_coupon,um_buy_rate,um_coupon_use_rate,um_buy_with_coupon_rate
0,1439408,2632,2,2,2,2,1.0,1.0,1.0
1,2029232,3381,1,1,1,1,1.0,1.0,1.0
2,2223968,3381,1,1,1,1,1.0,1.0,1.0
3,73611,2099,1,1,1,1,1.0,1.0,1.0
4,3273056,4833,1,1,1,1,1.0,1.0,1.0


### 將 user_feature, merchant_feature, user_merchant_feature 放入訓練集和測試集中

In [72]:
# add user_feature, merchant_feature, user_merchant_feature to train data 
def merge_new_features(df):
    df_2 = pd.merge(df, user_feature, on = 'User_id', how = 'left').fillna(0)
    df_3 = pd.merge(df_2, merchant_feature, on = 'Merchant_id', how = 'left').fillna(0)
    df_4 = pd.merge(df_3, user_merchant_feature, on = ['User_id','Merchant_id'], how = 'left').fillna(0)
    return df_4.copy()

In [73]:
df_train = merge_new_features(df_valid)
df_test = merge_new_features(df_test)

In [74]:
df_train.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_top_up,...,m_median_distance,m_coupon_use_rate,m_sale_with_coupon_rate,um_count,um_buy_count,um_coupon_count,um_buy_with_coupon,um_buy_rate,um_coupon_use_rate,um_buy_with_coupon_rate
0,1832624,3381,7610,200:20,0.0,20160429,nan,1,0.900000,200.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,163606,1569,5054,200:30,10.0,20160421,nan,1,0.850000,200.0,...,8.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4061024,3381,7610,200:20,10.0,20160426,nan,1,0.900000,200.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,106443,450,3732,30:5,nan,20160429,nan,1,0.833333,30.0,...,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,114747,1569,5054,200:30,9.0,20160426,nan,1,0.850000,200.0,...,8.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [75]:
df_test.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,discount_type,discount_rate,discount_top_up,discount_off,...,m_median_distance,m_coupon_use_rate,m_sale_with_coupon_rate,um_count,um_buy_count,um_coupon_count,um_buy_with_coupon,um_buy_rate,um_coupon_use_rate,um_buy_with_coupon_rate
0,1439408,4663,11002,150:20,1.0,20160528,1,0.866667,150.0,20.0,...,7.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1439408,2632,8591,20:1,0.0,20160613,1,0.950000,20.0,1.0,...,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0
2,1439408,2632,nan,nan,0.0,nan,-1,1.000000,0.0,0.0,...,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0
3,1439408,2632,8591,20:1,0.0,20160516,1,0.950000,20.0,1.0,...,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0
4,2029232,450,1532,30:5,0.0,20160530,1,0.833333,30.0,5.0,...,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 所有特徵

In [76]:
predictors = original_feature + user_feature.columns.tolist()[1:] + \
             merchant_feature.columns.tolist()[1:] + \
             user_merchant_feature.columns.tolist()[2:]
print(len(predictors),predictors)

41 ['discount_rate', 'discount_type', 'discount_top_up', 'discount_off', 'distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7', 'u_coupon_count', 'u_buy_count', 'u_buy_with_coupon', 'u_merchant_count', 'u_min_distance', 'u_max_distance', 'u_mean_distance', 'u_median_distance', 'u_use_coupon_rate', 'u_buy_with_coupon_rate', 'm_coupon_count', 'm_sale_count', 'm_sale_with_coupon', 'm_user_count', 'm_min_distance', 'm_max_distance', 'm_mean_distance', 'm_median_distance', 'm_coupon_use_rate', 'm_sale_with_coupon_rate', 'um_count', 'um_buy_count', 'um_coupon_count', 'um_buy_with_coupon', 'um_buy_rate', 'um_coupon_use_rate', 'um_buy_with_coupon_rate']


## 劃分訓練集和驗證集

In [77]:
trainSub, validSub = train_test_split(df_train, test_size = 0.2, stratify = df_train['label'], random_state=100)

In [78]:
X_train = trainSub[predictors]
Y_train = trainSub['label']

In [79]:
model = check_model_SGDClassifier(X_train, Y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    1.7s finished


In [80]:
check_valid(model, validSub, predictors)

df_submit = process_pred(model, df_test, predictors)
to_csv(df_submit, 'submission_SGDClassifier.csv')
save_model(model, 'SGDClassifier_model.pkl')

Validation AUC: 0.827, Accuracy: 0.952
AvgAUC: 0.5857207273695335


# Blending

In [81]:
def process_pred_blending(model, df, predicts):
    df_ = pred_test(model, df, predicts)
    df_ = edit_proper_column(df_)
    return df_

In [82]:
lr = check_model_LogisticRegression(X_train, Y_train)
check_valid(lr, validSub, predictors)
df_lr = process_pred_blending(lr, df_test, predictors)
df_submit_lr = keep_higher_probability_if_same_uid(df_lr)
to_csv(df_submit_lr, 'submission_LogisticRegression.csv')

gdbt = check_model_GradientBoostingClassifier(X_train, Y_train)
check_valid(gdbt, validSub, predictors)
df_gdbt = process_pred_blending(gdbt, df_test, predictors)
df_submit_gdbt = keep_higher_probability_if_same_uid(df_gdbt)
to_csv(df_submit_gdbt, 'submission_GradientBoostingClassifier.csv')

rf = check_model_RandomForestClassifier(X_train, Y_train)
check_valid(rf, validSub, predictors)
df_rf = process_pred_blending(rf, df_test, predictors)
df_submit_rf = keep_higher_probability_if_same_uid(df_rf)
to_csv(df_submit_rf, 'submission_RandomForestClassifier.csv')

Validation AUC: 0.798, Accuracy: 0.951
AvgAUC: 0.5172097494322389
Validation AUC: 0.878, Accuracy: 0.955
AvgAUC: 0.6063120273213835
Validation AUC: 0.863, Accuracy: 0.953
AvgAUC: 0.6196203715535419


In [83]:
# submission_LogisticRegression: Kaggle 0.66104
# submission_GradientBoostingClassifier: Kaggle 0.69956
# submission_RandomForestClassifier: Kaggle 0.69559

blending_pred = df_lr['label']*0.03  + df_gdbt['label']*0.57 + df_rf['label']*0.4
df_submit_rf['label'] = blending_pred
df_submit = keep_higher_probability_if_same_uid(df_submit_rf)
to_csv(df_submit, 'submission_Blending.csv')

# Stacking

In [84]:
model = check_model_StackingClassifier(X_train, Y_train)

In [85]:
check_valid(model, validSub, predictors)
df_submit = process_pred(model, df_test, predictors)
to_csv(df_submit, 'submission_StackingClassifier.csv')

Validation AUC: 0.547, Accuracy: 0.955
AvgAUC: 0.5155159155865846


# 更多feature參考

In [86]:
# 總用戶 u0
# 總商家 m0
# 總優惠券種量 d0


# 用戶特徵：

# 用戶一共消費多少次 u1
# 線下領取優惠卷的次數 u2
# 線下領取優惠券但沒有使用的次數 u3
# 線下領取優惠卷並核銷的次數 u4
# 線下普通消費次數 u5
# 線下領取優惠券到核銷的平均間隔時間 u6
# 線下正常消費的平均間隔時間 u7
# 最近一次優惠券消費到當前領券的時間間隔 u8
# 最近一次普通消費到當前領券的時間間隔 u9
# 用戶核銷優惠券的平均消費折率 u10
# 用戶核銷優惠券的最低消費折率 u11
# 用戶核銷優惠券的最高消費折率 u12
# 用戶核銷優惠券中的平均與商家距離 u13
# 用戶核銷優惠券中的最小與商家距離 u14
# 用戶核銷優惠券中的最大與商家距離 u15
# 用戶核銷過的不同優惠券種量 u16
# 用戶核銷幾個商家 u17
# 用戶平均核銷每個商家多少張優惠券 u18
# 15/u6 用戶15天內平均使用優惠卷消費幾次 u19
# 15/u7 用戶15天內平均會普通消費幾次 u20
# u16/d1 用戶核銷過的不同優惠券數量佔所有不同優惠券的比重 u21


# 商家特徵：

# 商家一共消費次數 m1
# 商家優惠券被領取後不核銷次數 m2
# 商家優惠券被領取後核銷次數 m3
# m2+m3 商家優惠券被領取次數 m4
# 商家普通消費次數 m5
# 商家優惠券核銷的平均消費折率 m6
# 商家優惠券核銷的最小消費折率 m7
# 商家優惠券核銷的最大消費折率 m8
# 核銷商家優惠券的不同用戶數量 m9
# 商家被核銷過的不同優惠券數量 m10
# 商家不同優惠券數量 m11
# 商家平均每種優惠券核銷多少張 m12
# 商家被核銷優惠券的平均時間 m13
# 商家被核銷優惠券中的平均距離 m14
# 商家被核銷優惠券中的最小距離 m15
# 商家被核銷優惠券中的最大距離 m16
# 商家發行的優惠券數目 m17
# 商家有多少人在此店領券 m18
# m3/m4 商家優惠券被領取後核銷率 m19
# m3/m19 商家優惠券平均每個用戶核銷多少張 m20
# m10/m11 商家被核銷過的不同優惠券數量佔所有領取過的不同優惠券數量的比重 m21


# 優惠卷特徵：

# 一共發行多少張 d1
# 沒有使用的數目 d2
# 核銷多少張 d3
# 優惠券類型(直接優惠為0, 滿減為1) d4
# 滿減類優惠券滿減金額 d5
# 滿減類優惠券減的金額 d6
# 歷史上用戶領取該優惠券次數 d7
# 歷史上用戶核銷該優惠券次數 d8
# 領取優惠券是一周的第幾天 d9
# 領取優惠券是一月的第幾天 d10
# d3/d1 歷史上用戶對該優惠券的核銷率 d11
# d6/d5 優惠券折率 d12
# 當天所領取優惠券裡面優惠券折率排名 d13


# 用戶-商家組合特徵：

# 用戶在商家總共消費過幾次 um1
# 用戶領取商家的優惠券次數 um2
# 用戶領取商家的優惠券後不核銷次數 um3
# 用戶領取商家的優惠券後核銷次數 um4
# 用戶在商家普通消費次數 um5
# um4/um2 用戶領取商家的優惠券後核銷率 um6
# um1/(u4+u5) 用戶常去商家 um7
# um4/u4 用戶常在商家使用優惠卷 um8


# 其它特徵，這部分特徵利用了賽題leakage，都是在預測區間提取的：

# 預測集，商家發行的優惠券數目 m22
# 預測集，商家有多少人在此店領券 m23
# 預測集，用戶此次之前領取的所有優惠券數目 u22
# 預測集，用戶此次之後領取的所有優惠券數目 u23
# 預測集，用戶領取的所有優惠券數目 u24
# 預測集，用戶領取的特定優惠券數目 u25
# 預測集，用戶上一次領取的時間間隔 u26
# 預測集，用戶上下一次領取的時間間隔 u27